Você recebeu uma amostra do banco de dados da HW para que possa tratá-la de maneira adequada.

-	Essa é uma amostra da tabela de transações, que será utilizada para futuras análises na HW (transactions.csv). 
-	Além dela, você também tem acesso à tabela de bandeira do cartão (card_brand.csv), com a relação de/para do código com o nome da bandeira. 


Objetivo:

Você deve criar um (ou mais) script (de preferência em Python) para tratar essa base para que a equipe de negócio possa analisá-la com menor esforço possível. 

Você pode fazer todo e qualquer tratamento que achar necessário, como substituição de valores, criação/deleção de colunas, alteração do nome de colunas.

In [86]:
import pandas as pd

In [87]:
transactions = pd.read_csv('transactions.csv', sep=";")

In [88]:
transactions.head()

,id,created_at,merchant_id,valor,n_PARCELAS,Nome_no_Cartao,status,card_id,iso_id,card_brand,DOCUMENTO
0,2223391,2021-01-10 00:01:13.555+00,5bc62411f19d470007acbc8c,9474,1,Ana Luísa souza oliveira,Paid,560555555,62,100.0,510574177-567
1,2223371,2021-01-10 00:00:39.953+00,5c0fa52815f3dd0007bb1e16,2742,1,antônio Silva,refused,5555551552,9,456.0,717355279-706
2,2223376,2021-01-10 00:00:52.936+00,5bfd833c998a600007b27f93,2235,1,FELIPE FERREIRA,in process,225440550,9,100.0,339057504526
3,2223379,2021-01-10 00:00:56.639+00,5bab6f250cb37a00075f12f9,3104,1,antônio oliveira,Paid,550000540,62,100.0,NaN
4,2223397,2021-01-10 00:05:08.929+00,5bd9cece05e0ce0007f9161a,11431,1,RAFAEL Pereira,paid,1001020,35,456.0,083.497.797-797


In [89]:
card_brand = pd.read_csv('card_brand.csv', sep=",")
card_brand

,id,brand_name,brand_code
0,1,visa,100
1,2,mastercard,456
2,3,elo,903
3,4,alelo,1020
4,5,ticket,857
5,6,sodexo,2343
6,7,hiper,392


In [90]:
# Add record to handle NaN values in Transactions data car_brand column
new_row = {'id': 8, 'brand_name': 'None', 'brand_code': 0}
card_brand = card_brand.append(new_row, ignore_index = True)
card_brand

,id,brand_name,brand_code
0,1,visa,100
1,2,mastercard,456
2,3,elo,903
3,4,alelo,1020
4,5,ticket,857
5,6,sodexo,2343
6,7,hiper,392
7,8,None,0


In [91]:
# Create a column to merge transactions and time_dimension
transactions['date'] = pd.to_datetime(transactions['created_at'],infer_datetime_format=True).dt.date

In [92]:
min_date = min(transactions['date'])
max_date = max(transactions['date'])

print(min_date)
print(max_date)

2021-01-10
2021-01-12


In [93]:
def create_date_table(start=f'{min_date}', end=f'{max_date}'):
    df = pd.DataFrame({"date": pd.date_range(start, end)})
    df["week_day"] = df.date.dt.day_name()
    df["day"] = df.date.dt.day
    df["month"] = df.date.dt.month
    df["week"] = df.date.dt.isocalendar().week
    df["quarter"] = df.date.dt.quarter
    df["year"] = df.date.dt.year
    df.insert(0, 'date_id', (df.year.astype(str) + df.month.astype(str).str.zfill(2) + df.day.astype(str).str.zfill(2)).astype(int))
    return df

df_datetime_dimension = create_date_table()

In [94]:
df_datetime_dimension.head()

,date_id,date,week_day,day,month,week,quarter,year
0,20210110,2021-01-10,Sunday,10,1,1,1,2021
1,20210111,2021-01-11,Monday,11,1,2,1,2021
2,20210112,2021-01-12,Tuesday,12,1,2,1,2021


In [95]:
transactions['date'] = pd.to_datetime(transactions['date'])
# Fazendo merge do dataframe Fato com a Dimensão Tempo
transactions_time = pd.merge(transactions, df_datetime_dimension, on='date', how='inner')

In [96]:
transactions_time.head()

,id,created_at,merchant_id,valor,n_PARCELAS,Nome_no_Cartao,status,card_id,iso_id,card_brand,DOCUMENTO,date,date_id,week_day,day,month,week,quarter,year
0,2223391,2021-01-10 00:01:13.555+00,5bc62411f19d470007acbc8c,9474,1,Ana Luísa souza oliveira,Paid,560555555,62,100.0,510574177-567,2021-01-10,20210110,Sunday,10,1,1,1,2021
1,2223371,2021-01-10 00:00:39.953+00,5c0fa52815f3dd0007bb1e16,2742,1,antônio Silva,refused,5555551552,9,456.0,717355279-706,2021-01-10,20210110,Sunday,10,1,1,1,2021
2,2223376,2021-01-10 00:00:52.936+00,5bfd833c998a600007b27f93,2235,1,FELIPE FERREIRA,in process,225440550,9,100.0,339057504526,2021-01-10,20210110,Sunday,10,1,1,1,2021
3,2223379,2021-01-10 00:00:56.639+00,5bab6f250cb37a00075f12f9,3104,1,antônio oliveira,Paid,550000540,62,100.0,NaN,2021-01-10,20210110,Sunday,10,1,1,1,2021
4,2223397,2021-01-10 00:05:08.929+00,5bd9cece05e0ce0007f9161a,11431,1,RAFAEL Pereira,paid,1001020,35,456.0,083.497.797-797,2021-01-10,20210110,Sunday,10,1,1,1,2021


In [97]:
# Replace Nan Values in card_brand to Number 0
transactions_time['card_brand'] = transactions_time['card_brand'].fillna(0)

# Merge transactions and time dataframe with card_brand dataframe by card_brand and brand_code
transactions_time_card = pd.merge(transactions_time, card_brand, left_on='card_brand', right_on='brand_code', how='inner')

In [98]:
transactions_time.isnull().sum()

id                   0
created_at           0
merchant_id          0
valor                0
n_PARCELAS           0
Nome_no_Cartao    4449
status               0
card_id              0
iso_id               0
card_brand           0
DOCUMENTO         4565
date                 0
date_id              0
week_day             0
day                  0
month                0
week                 0
quarter              0
year                 0
dtype: int64

In [99]:
transactions_time_card.head()

,id_x,created_at,merchant_id,valor,n_PARCELAS,Nome_no_Cartao,status,card_id,iso_id,card_brand,DOCUMENTO,date,date_id,week_day,day,month,week,quarter,year,id_y,brand_name,brand_code
0,2223391,2021-01-10 00:01:13.555+00,5bc62411f19d470007acbc8c,9474,1,Ana Luísa souza oliveira,Paid,560555555,62,100.0,510574177-567,2021-01-10,20210110,Sunday,10,1,1,1,2021,1,visa,100
1,2223376,2021-01-10 00:00:52.936+00,5bfd833c998a600007b27f93,2235,1,FELIPE FERREIRA,in process,225440550,9,100.0,339057504526,2021-01-10,20210110,Sunday,10,1,1,1,2021,1,visa,100
2,2223379,2021-01-10 00:00:56.639+00,5bab6f250cb37a00075f12f9,3104,1,antônio oliveira,Paid,550000540,62,100.0,NaN,2021-01-10,20210110,Sunday,10,1,1,1,2021,1,visa,100
3,2223399,2021-01-10 00:05:08.988+00,5ca3a42f15751a0007e6c52b,10371,1,Jéssica lima,Paid,3383341477,35,100.0,484177507602,2021-01-10,20210110,Sunday,10,1,1,1,2021,1,visa,100
4,2223366,2021-01-10 00:00:09.209+00,5bd9cece05e0ce0007f9161a,57671,1,NaN,in process,8822882888,47,100.0,806674693887,2021-01-10,20210110,Sunday,10,1,1,1,2021,1,visa,100


In [100]:
# Remove unecessary and sensitive data columns, rename id_x to id, transform column names in lowercase, status, name and brand name to uppercase and valor to float

df_clean_columns = transactions_time_card.drop(['date_id', 'id_y', 'card_brand', 'DOCUMENTO'], axis=1)
df_clean_columns = df_clean_columns.rename(columns={'id_x': 'id'})
df_clean_columns.columns = df_clean_columns.columns.str.lower()
df_clean_columns['nome_no_cartao'] = df_clean_columns['nome_no_cartao'].str.upper()
df_clean_columns['status'] = df_clean_columns['status'].str.upper()
df_clean_columns['brand_name'] = df_clean_columns['brand_name'].str.upper()
df_clean_columns['valor'] = df_clean_columns['valor'].apply(lambda x: format(float(x),".2f"))
df_clean_columns.head()

,id,created_at,merchant_id,valor,n_parcelas,nome_no_cartao,status,card_id,iso_id,date,week_day,day,month,week,quarter,year,brand_name,brand_code
0,2223391,2021-01-10 00:01:13.555+00,5bc62411f19d470007acbc8c,9474.00,1,ANA LUÍSA SOUZA OLIVEIRA,PAID,560555555,62,2021-01-10,Sunday,10,1,1,1,2021,VISA,100
1,2223376,2021-01-10 00:00:52.936+00,5bfd833c998a600007b27f93,2235.00,1,FELIPE FERREIRA,IN PROCESS,225440550,9,2021-01-10,Sunday,10,1,1,1,2021,VISA,100
2,2223379,2021-01-10 00:00:56.639+00,5bab6f250cb37a00075f12f9,3104.00,1,ANTÔNIO OLIVEIRA,PAID,550000540,62,2021-01-10,Sunday,10,1,1,1,2021,VISA,100
3,2223399,2021-01-10 00:05:08.988+00,5ca3a42f15751a0007e6c52b,10371.00,1,JÉSSICA LIMA,PAID,3383341477,35,2021-01-10,Sunday,10,1,1,1,2021,VISA,100
4,2223366,2021-01-10 00:00:09.209+00,5bd9cece05e0ce0007f9161a,57671.00,1,NaN,IN PROCESS,8822882888,47,2021-01-10,Sunday,10,1,1,1,2021,VISA,100


In [101]:
# Delete sensitive data
del df_clean_columns['nome_no_cartao']

In [102]:
df_clean_columns.head()

,id,created_at,merchant_id,valor,n_parcelas,status,card_id,iso_id,date,week_day,day,month,week,quarter,year,brand_name,brand_code
0,2223391,2021-01-10 00:01:13.555+00,5bc62411f19d470007acbc8c,9474.00,1,PAID,560555555,62,2021-01-10,Sunday,10,1,1,1,2021,VISA,100
1,2223376,2021-01-10 00:00:52.936+00,5bfd833c998a600007b27f93,2235.00,1,IN PROCESS,225440550,9,2021-01-10,Sunday,10,1,1,1,2021,VISA,100
2,2223379,2021-01-10 00:00:56.639+00,5bab6f250cb37a00075f12f9,3104.00,1,PAID,550000540,62,2021-01-10,Sunday,10,1,1,1,2021,VISA,100
3,2223399,2021-01-10 00:05:08.988+00,5ca3a42f15751a0007e6c52b,10371.00,1,PAID,3383341477,35,2021-01-10,Sunday,10,1,1,1,2021,VISA,100
4,2223366,2021-01-10 00:00:09.209+00,5bd9cece05e0ce0007f9161a,57671.00,1,IN PROCESS,8822882888,47,2021-01-10,Sunday,10,1,1,1,2021,VISA,100


In [103]:
!pip install nbconvert